In [1]:
import numpy as np
import pandas as pd
from subprocess import check_output
from sklearn import ensemble, metrics, linear_model
import random
%matplotlib inline

In [2]:
class addNearestNeighbourLinearFeatures:
    
    def __init__(self, n_neighbours=1, max_elts=None, verbose=True, random_state=None):
        self.rnd = random_state
        self.n = n_neighbours
        self.max_elts = max_elts
        self.verbose = verbose
        self.neighbours = []
        self.clfs = []
        
    def fit(self, train, y):
        if self.rnd != None:
            random.seed(rnd)
        if self.max_elts == None:
            self.max_elts = len(train.columns)
        list_vars = list(train.columns)
        random.shuffle(list_vars)
        
        lastscores = np.zeros(self.n) + 1e15

        for elt in list_vars[:self.n]:
            self.neighbours.append([elt])
        list_vars = list_vars[self.n:]
        
        for elt in list_vars:
            indice = 0
            scores = []
            for elt2 in self.neighbours:
                if len(elt2) < self.max_elts:
                    clf = linear_model.LinearRegression(fit_intercept=False, 
                                                        normalize=True, copy_X=True) 
                    clf.fit(train[elt2+[elt]], y)
                    scores.append(metrics.log_loss(y,clf.predict(train[elt2 + [elt]])))
                    indice=indice+1
                else:
                    scores.append(lastscores[indice])
                    indice=indice+1
            gains = lastscores - scores
            if gains.max() > 0:
                temp = gains.argmax()
                lastscores[temp] = scores[temp]
                self.neighbours[temp].append(elt)

        indice = 0
        for elt in self.neighbours:
            clf = linear_model.LinearRegression(fit_intercept=False, normalize=True, copy_X=True) 
            clf.fit(train[elt], y)
            self.clfs.append(clf)
            if self.verbose:
                print(indice, lastscores[indice], elt)
            indice = indice + 1
                    
    def transform(self, train):
        indice = 0
        for elt in self.neighbours:
            train['_'.join(pd.Series(elt).sort_values().values)] = self.clfs[indice].predict(train[elt])
            indice = indice + 1
        return train
    
    def fit_transform(self, train, y):
        self.fit(train, y)
        return self.transform(train)

In [3]:
train = pd.read_csv("data/train.csv")
target = train['target'].values
test = pd.read_csv("data/test.csv")
id_test = test['ID'].values

In [4]:
train['v22-1']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
test['v22-1']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
train['v22-2']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
test['v22-2']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
train['v22-3']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
test['v22-3']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
train['v22-4']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))
test['v22-4']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))

drop_list=['v91','v1', 'v8', 'v10', 'v15', 'v17', 'v25', 'v29', 'v34', 'v41', 'v46', 'v54', 'v64', 'v67', 'v97', 'v105', 'v111', 'v122']
train = train.drop(['ID','target'] + drop_list,axis=1).fillna(-999)
test = test.drop(['ID'] + drop_list,axis=1).fillna(-999)

refcols=list(train.columns)

In [5]:
for elt in refcols:
    if train[elt].dtype=='O':
        train[elt], temp = pd.factorize(train[elt])
        test[elt]=temp.get_indexer(test[elt])
    else:
        train[elt]=train[elt].round(5)
        test[elt]=test[elt].round(5)

In [6]:
rnd=12
random.seed(rnd)

In [7]:
from sklearn.cross_validation import StratifiedKFold

In [8]:
skf = StratifiedKFold(target, n_folds=4)

In [9]:
from sklearn.metrics import log_loss

In [10]:
from libtelepot import sendMessage

In [11]:
from hyperopt import fmin, hp, tpe, STATUS_OK

In [12]:
def objective(space):
    n_ft = int(space['n_ft'])
    max_elts = int(space['max_elts'])
    #n_ft = max_elts * 3
    
    scores = list()
    
    for train_index, test_index in skf:
        X_train, X_test = train.iloc[train_index].copy(), train.iloc[test_index].copy()
        y_train, y_test = target[train_index], target[test_index]
        a = addNearestNeighbourLinearFeatures(n_neighbours=n_ft, max_elts=max_elts,
                                              verbose=False, random_state=rnd)
        a.fit(X_train, y_train)
        X_train = a.transform(X_train)
        X_test = a.transform(X_test)
        
        clf = ensemble.ExtraTreesClassifier(n_estimators=500, max_features=int(space['max_features']),
                                            criterion='entropy', min_samples_split=4, 
                                            max_depth=int(space['max_depth']), min_samples_leaf=2, 
                                            n_jobs=-1, random_state=rnd)
        clf.fit(X_train, y_train)
        pred_et = clf.predict_proba(X_test)[:, 1]
        scores.append(log_loss(y_test, pred_et))
    
    sendMessage('Scores for space {:} are {:.5f} +/- {:.5f}'.format(space, np.mean(scores), np.std(scores)))
    return {'loss': np.mean(scores), 'loss_variance': np.std(scores), 'status': STATUS_OK}

In [13]:
space = {'max_depth': hp.quniform('max_depth', 25, 70, 5),
         'max_features': hp.quniform('max_features', 30, 100, 5),
         'n_ft': hp.quniform('n_ft', 10, 60, 5),
         'max_elts': hp.quniform('max_elts', 4, 20, 2)
}

In [ ]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=50)

In [2]:

n_ft=40 #Number of features to add
max_elts=5

In [7]:
a=addNearestNeighbourLinearFeatures(n_neighbours=n_ft, max_elts=max_elts, verbose=True, random_state=rnd)
a.fit(train, target)

train = a.transform(train)
test = a.transform(test)

0 0.564640677692 ['v95', 'v22-3']
1 0.78492204104 ['v93', 'v39', 'v55', 'v45']
2 1.0150421579 ['v103', 'v32']
3 1.2237846517 ['v112', 'v30', 'v38', 'v90', 'v100']
4 0.695421552089 ['v77', 'v18']
5 1.51026055801 ['v12', 'v52', 'v24']
6 0.912545995271 ['v131', 'v69']
7 0.590485761075 ['v50', 'v14']
8 1.27423497001 ['v71', 'v51', 'v107', 'v129', 'v85']
9 1.34909645163 ['v82', 'v89', 'v70']
10 0.953489101882 ['v128', 'v114']
11 0.597755677821 ['v102', 'v110', 'v11', 'v35', 'v121']
12 0.67363493342 ['v6', 'v49']
13 0.560716786825 ['v43', 'v53']
14 0.55909244045 ['v47', 'v22-4']
15 1.02753943928 ['v62', 'v31']
16 1.71151478424 ['v40', 'v58']
17 0.586971031973 ['v116', 'v20']
18 0.672166410312 ['v5', 'v108', 'v81']
19 0.549512531659 ['v22-1', 'v109']
20 0.74920712197 ['v61', 'v127', 'v101']
21 0.777851399127 ['v117', 'v23', 'v27', 'v126', 'v94']
22 0.933721028047 ['v92', 'v28', 'v72', 'v3']
23 0.63493438842 ['v88', 'v68']
24 0.661909492268 ['v65', 'v26']
25 0.731832815015 ['v115', 'v76', 'v33

In [ ]:
params = [{'max_features': 50, 'max_depth': 35, 'random'}]

In [9]:
preds = list()
for mf in [50, 60, 70]:
    for md in [35, 40, 45]:
        for rnd in [0, 12, 42]:
            print(rnd, md, mf, flush=True)
            clf = ensemble.ExtraTreesClassifier(n_estimators=3750, max_features=mf ,criterion='entropy',
                                                min_samples_split=4, max_depth=md, min_samples_leaf=2, 
                                                n_jobs = -1, random_state=rnd)
            clf.fit(train,target)
            pred_et = clf.predict_proba(test)[:, 1]
            preds.append(pred_et)

0 35 50
12 35 50
42 35 50
0 40 50
12 40 50
42 40 50
0 45 50
12 45 50
42 45 50
0 35 60
12 35 60
42 35 60
0 40 60
12 40 60
42 40 60
0 45 60
12 45 60
42 45 60
0 35 70
12 35 70
42 35 70
0 40 70
12 40 70
42 40 70
0 45 70
12 45 70
42 45 70


In [12]:
np.array(preds).mean(axis=0).shape

(114393,)

In [14]:
submission=pd.read_csv('data/sample_submission.csv')
submission.index=submission.ID
submission.PredictedProb=np.array(preds).mean(axis=0)
submission.to_csv('submission/et_ln_bag1.csv', index=False)
submission.PredictedProb.hist(bins=30)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

 # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from sklearn import ensemble, metrics, linear_model
import random

#Some parameters to play with
rnd=12
random.seed(rnd)
n_ft=20 #Number of features to add
max_elts=3 #Maximum size of a group of linear features

class addNearestNeighbourLinearFeatures:
    
    def __init__(self, n_neighbours=1, max_elts=None, verbose=True, random_state=None):
        self.rnd=random_state
        self.n=n_neighbours
        self.max_elts=max_elts
        self.verbose=verbose
        self.neighbours=[]
        self.clfs=[]
        
    def fit(self,train,y):
        if self.rnd!=None:
            random.seed(rnd)
        if self.max_elts==None:
            self.max_elts=len(train.columns)
        list_vars=list(train.columns)
        random.shuffle(list_vars)
        
        lastscores=np.zeros(self.n)+1e15

        for elt in list_vars[:self.n]:
            self.neighbours.append([elt])
        list_vars=list_vars[self.n:]
        
        for elt in list_vars:
            indice=0
            scores=[]
            for elt2 in self.neighbours:
                if len(elt2)<self.max_elts:
                    clf=linear_model.LinearRegression(fit_intercept=False, normalize=True, copy_X=True, n_jobs=-1) 
                    clf.fit(train[elt2+[elt]], y)
                    scores.append(metrics.log_loss(y,clf.predict(train[elt2 + [elt]])))
                    indice=indice+1
                else:
                    scores.append(lastscores[indice])
                    indice=indice+1
            gains=lastscores-scores
            if gains.max()>0:
                temp=gains.argmax()
                lastscores[temp]=scores[temp]
                self.neighbours[temp].append(elt)

        indice=0
        for elt in self.neighbours:
            clf=linear_model.LinearRegression(fit_intercept=False, normalize=True, copy_X=True, n_jobs=-1) 
            clf.fit(train[elt], y)
            self.clfs.append(clf)
            if self.verbose:
                print(indice, lastscores[indice], elt)
            indice=indice+1
                    
    def transform(self, train):
        indice=0
        for elt in self.neighbours:
            train['_'.join(pd.Series(elt).sort_values().values)]=self.clfs[indice].predict(train[elt])
            indice=indice+1
        return train
    
    def fit_transform(self, train, y):
        self.fit(train, y)
        return self.transform(train)
    
    
train = pd.read_csv("../input/train.csv")
target = train['target'].values
test = pd.read_csv("../input/test.csv")
id_test = test['ID'].values

train['v22-1']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
test['v22-1']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[0]))
train['v22-2']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
test['v22-2']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[1]))
train['v22-3']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
test['v22-3']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[2]))
train['v22-4']=train['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))
test['v22-4']=test['v22'].fillna('@@@@').apply(lambda x:'@'*(4-len(str(x)))+str(x)).apply(lambda x:ord(x[3]))

drop_list=['v91','v1', 'v8', 'v10', 'v15', 'v17', 'v25', 'v29', 'v34', 'v41', 'v46', 'v54', 'v64', 'v67', 'v97', 'v105', 'v111', 'v122']
train = train.drop(['ID','target'] + drop_list,axis=1).fillna(-999)
test = test.drop(['ID'] + drop_list,axis=1).fillna(-999)

refcols=list(train.columns)

for elt in refcols:
    if train[elt].dtype=='O':
        train[elt], temp = pd.factorize(train[elt])
        test[elt]=temp.get_indexer(test[elt])
    else:
        train[elt]=train[elt].round(5)
        test[elt]=test[elt].round(5)
        
a=addNearestNeighbourLinearFeatures(n_neighbours=n_ft, max_elts=max_elts, verbose=True, random_state=rnd)
a.fit(train, target)

train = a.transform(train)
test = a.transform(test)

clf = ensemble.ExtraTreesClassifier(n_estimators=750,max_features=50,criterion= 'entropy',min_samples_split= 4,
                        max_depth= 35, min_samples_leaf= 2, n_jobs = -1, random_state=rnd)

clf.fit(train,target)
pred_et=clf.predict_proba(test)

submission=pd.read_csv('../input/sample_submission.csv')
submission.index=submission.ID
submission.PredictedProb=pred_et[:,1]
submission.to_csv('./addNNLinearFt.csv', index=False)
submission.PredictedProb.hist(bins=30)